In [12]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import ast

In [13]:

# Inicialización de MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=True, min_detection_confidence=0.5)

# Directorio que contiene los videos.
video_directory = 'D:/Codes/EFBA/DataEfba/videos'
video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if os.path.isfile(os.path.join(video_directory, f))]
excel_directory = 'D:/Codes/EFBA/DataEfba/keypoints'

# Procesar cada video en el directorio.
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    keypoints_list = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Conversión del color del frame de BGR a RGB.
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesamiento del frame para detectar la pose.
        results = pose.process(frame_rgb)
        
        if results.pose_landmarks:
            # Extracción de las coordenadas de los puntos clave.
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.append((landmark.x, landmark.y, landmark.z, landmark.visibility))
            keypoints_list.append(keypoints)

    cap.release()
    
    # Opcional: Convertir la lista de puntos clave a un DataFrame de pandas para un análisis más fácil.
    df_keypoints = pd.DataFrame(keypoints_list, columns=[f'x{i}' for i in range(11)] + [f'y{i}' for i in range(11)] + [f'z{i}' for i in range(11)])
    
    # Aquí puedes guardar o procesar df_keypoints como prefieras.
    # Por ejemplo, guardar en un archivo CSV.
    output_filename = os.path.splitext(os.path.basename(video_file))[0] + '_keypoints.csv'
    df_keypoints.to_csv(os.path.join(excel_directory, output_filename), index=False)

    print(f"Procesado {video_file}, resultados guardados en {output_filename}.")


Procesado D:/Codes/EFBA/DataEfba/videos\allen.mp4, resultados guardados en allen_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\bird.mp4, resultados guardados en bird_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\curry1.mp4, resultados guardados en curry1_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\curry2.mp4, resultados guardados en curry2_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\harden.mp4, resultados guardados en harden_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\jj.mp4, resultados guardados en jj_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\jordan.mp4, resultados guardados en jordan_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\kd.mp4, resultados guardados en kd_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\kerr.mp4, resultados guardados en kerr_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\klay.mp4, resultados guardados en klay_keypoints.csv.
Procesado D:/Codes/EFBA/DataEfba/videos\korver.mp4

In [15]:
def process_column(data):
    try:
        # Usa ast.literal_eval para convertir la cadena a una tupla
        return pd.DataFrame([ast.literal_eval(x) for x in data])
    except ValueError:
        # En caso de error, regresa un DataFrame vacío
        return pd.DataFrame()

def process_csv(file_path):
    df = pd.read_csv(file_path)
    
    # Preparar un diccionario para recopilar las nuevas columnas
    new_data = {}
    
    # Iterar sobre las columnas originales y procesar cada una
    for column in df.columns:
        # Si la columna es de tipo object (cadena), procesar su contenido
        if df[column].dtype == object:
            processed_data = process_column(df[column])
            # Si el procesamiento fue exitoso y el DataFrame no está vacío
            if not processed_data.empty:
                # Crear nuevas columnas para 'x', 'y', 'z' y 'visibility'
                new_data[f'{column}_x'] = processed_data[0]
                new_data[f'{column}_y'] = processed_data[1]
                new_data[f'{column}_z'] = processed_data[2]
                new_data[f'{column}_visibility'] = processed_data[3]
    
    # Crear un nuevo DataFrame con las nuevas columnas
    new_df = pd.DataFrame(new_data)
    
    # Guardar el nuevo DataFrame en un archivo CSV
    output_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.csv', '_processed.csv'))
    new_df.to_csv(output_file_path, index=False)
    print(f"Archivo procesado y guardado: {output_file_path}")

# Define el directorio donde se encuentran tus archivos CSV
directory_path = 'D:/Codes/EFBA/DataEfba/keypoints'
output_directory = 'D:/Codes/EFBA/DataEfba/pkeypoints'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Lista todos los archivos CSV en el directorio
csv_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.csv')]

# Procesa cada archivo CSV en el directorio
for file in csv_files:
    process_csv(file)


Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\allen_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\bird_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\curry1_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\curry2_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\harden_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\jj_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\jordan_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\kd_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\kerr_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\klay_keypoints_processed.csv
Archivo procesado y guardado: D:/Codes/EFBA/DataEfba/pkeypoints\k

In [17]:
def normalize_features(file_path):
    df = pd.read_csv(file_path)
    feature_columns = ['y0_x', 'y0_y', 'y0_z', 'y0_visibility',
'y1_x', 'y1_y', 'y1_z', 'y1_visibility',
'y2_x', 'y2_y', 'y2_z', 'y2_visibility',
'y3_x', 'y3_y', 'y3_z', 'y3_visibility',
'y4_x', 'y4_y', 'y4_z', 'y4_visibility',
'y5_x', 'y5_y', 'y5_z', 'y5_visibility',
'y6_x', 'y6_y', 'y6_z', 'y6_visibility',
'y7_x', 'y7_y', 'y7_z', 'y7_visibility',
'y8_x', 'y8_y', 'y8_z', 'y8_visibility',
'y9_x', 'y9_y', 'y9_z', 'y9_visibility',
'y10_x', 'y10_y', 'y10_z', 'y10_visibility',
'z0_x', 'z0_y', 'z0_z', 'z0_visibility',
'z1_x', 'z1_y', 'z1_z', 'z1_visibility',
'z2_x', 'z2_y', 'z2_z', 'z2_visibility',
'z3_x', 'z3_y', 'z3_z', 'z3_visibility',
'z4_x', 'z4_y', 'z4_z', 'z4_visibility',
'z5_x', 'z5_y', 'z5_z', 'z5_visibility',
'z6_x', 'z6_y', 'z6_z', 'z6_visibility',
'z7_x', 'z7_y', 'z7_z', 'z7_visibility',
'z8_x', 'z8_y', 'z8_z', 'z8_visibility',
'z9_x', 'z9_y', 'z9_z', 'z9_visibility',
'z10_x', 'z10_y', 'z10_z', 'z10_visibility']
    features = df[feature_columns]
    features_normalized = (features -  features.mean())/features.std()
    df[feature_columns] =  features_normalized
    
    normalize_file_path =  file_path.replace('.csv','_normalized.csv')
    df.to_csv(normalize_file_path,index= False)
    print(f"Archvio normalizado guardado en: {normalize_file_path}")

folder_path = 'D:/Codes/EFBA/DataEfba/pkeypoints'

csv_files = [os.path.join(folder_path,f) for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in csv_files:
    normalize_features(file)

Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\allen_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\bird_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\curry1_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\curry2_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\harden_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\jj_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\jordan_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\kd_keypoints_processed_normalized.csv
Archvio normalizado guardado en: D:/Codes/EFBA/DataEfba/pkeypoints\kerr_keypoints_processed_normalized.csv
Archvio normalizado guardado en:

In [18]:
from sklearn.model_selection import train_test_split

In [19]:

# Suponiendo que 'features_normalized' son tus características y tienes una columna 'target' en tu DataFrame que representa la variable objetivo.
X = features_normalized  # Características
y = df['']  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Si también quieres un conjunto de validación, puedes dividir el conjunto de entrenamiento una vez más
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Ahora tienes:
# - 60% de los datos en el conjunto de entrenamiento
# - 20% de los datos en el conjunto de validación
# - 20% de los datos en el conjunto de prueba


KeyError: ''